In [11]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import WordPunctTokenizer 
import string 
import csv
import gensim
from gensim import corpora
import re
import pandas as pd
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

import numpy as np  
import pandas as pd 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 
from sklearn import cluster
import seaborn as sns

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brittanysteenbergen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brittanysteenbergen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brittanysteenbergen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
file = open('mathematics.txt', 'r')
lines = file.readlines()
lines = [line.lower() for line in lines]
for line in lines:
    print(line)

what are your plans after graduation?

would you like to provide a non-lmu email so we can contact you after graduation? you can also join in the lmu math department linkedin network.

student-faculty relationship and department environment: during your time in the math department, how do you feel about

your relation and interaction with faculty in the math department

student-student relationships in the department

advising: how do you feel about

academic program advising in mathematics

the way advisors are assigned

career advising

advising on personal matters if relevant

lower-division experience: what are your thoughts on

lower division course work in math 

the level of difficulty of lower division math courses 

transition from high school to university

non-course experiences, e.g. dorms, added freedom, etc.

upper division experience: what are your thoughts on

difficulty level of upper division math courses

variety and availability of upper division math courses

trans

In [13]:
# document processing using nltk
# https://towardsdatascience.com/text-normalization-with-spacy-and-nltk-1302ff430119

doc_complete = lines
tk = WordPunctTokenizer() 

def tokenizing_nltk(doc):
    # tokenize
    doc_tokenized = tk.tokenize(doc)

    # stemming
    # nltk_stemedList = []
    # for word in doc_tokenized:
    #     nltk_stemedList.append(p_stemmer.stem(word))
    # print(nltk_stemedList)

    # lematize
    wordnet_lemmatizer = WordNetLemmatizer()
    nltk_lemmaList = []
    for word in doc_tokenized:
        nltk_lemmaList.append(wordnet_lemmatizer.lemmatize(word))

    # filter stopword
    filtered_sentence = []  
    nltk_stop_words = set(stopwords.words("english"))
    my_stop_words = {"I", "4", "ha", "c", ")", ").", "|", "3", "-", "(","–", "e", "249"}
    
    for w in nltk_lemmaList:  
        if w not in nltk_stop_words and w not in my_stop_words:  
            filtered_sentence.append(w)

    # removing punctuation
    punctuations="?:!.,;-"
    for word in filtered_sentence:
        if word in punctuations:
            filtered_sentence.remove(word)

    return filtered_sentence

doc_tokenized = [tokenizing_nltk(doc) for doc in doc_complete] 
print(doc_tokenized)

[['plan', 'graduation'], ['would', 'like', 'provide', 'non', 'lmu', 'email', 'contact', 'graduation', 'also', 'join', 'lmu', 'math', 'department', 'linkedin', 'network'], ['student', 'faculty', 'relationship', 'department', 'environment', 'time', 'math', 'department', 'feel'], ['relation', 'interaction', 'faculty', 'math', 'department'], ['student', 'student', 'relationship', 'department'], ['advising', 'feel'], ['academic', 'program', 'advising', 'mathematics'], ['way', 'advisor', 'assigned'], ['career', 'advising'], ['advising', 'personal', 'matter', 'relevant'], ['lower', 'division', 'experience', 'thought'], ['lower', 'division', 'course', 'work', 'math'], ['level', 'difficulty', 'lower', 'division', 'math', 'course'], ['transition', 'high', 'school', 'university'], ['non', 'course', 'experience', 'g', 'dorm', 'added', 'freedom', 'etc', '.'], ['upper', 'division', 'experience', 'thought'], ['difficulty', 'level', 'upper', 'division', 'math', 'course'], ['variety', 'availability', '

In [14]:
documents = [' '.join(tokens) for tokens in doc_tokenized]
print(documents)

['plan graduation', 'would like provide non lmu email contact graduation also join lmu math department linkedin network', 'student faculty relationship department environment time math department feel', 'relation interaction faculty math department', 'student student relationship department', 'advising feel', 'academic program advising mathematics', 'way advisor assigned', 'career advising', 'advising personal matter relevant', 'lower division experience thought', 'lower division course work math', 'level difficulty lower division math course', 'transition high school university', 'non course experience g dorm added freedom etc .', 'upper division experience thought', 'difficulty level upper division math course', 'variety availability upper division math course', 'transition lower division upper division math see mention math', 'feel versus unit math course', 'feel unit course across campus unit course time better 5 unit course', 'feel ability confidence learn mathematics', 'feel abil

In [15]:
# https://www.geeksforgeeks.org/clustering-text-documents-using-k-means-in-scikit-learn/

vectorizer = TfidfVectorizer(stop_words='english') 
  
# vectorize the text documents 
vectorized_documents = vectorizer.fit_transform(documents) 

final_df = pd.DataFrame(data = vectorized_documents.toarray(), columns=vectorizer.get_feature_names_out())

print("{} rows".format(final_df.shape[0]))
final_df.T.nlargest(5, 0)

26 rows


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
plan,0.747942,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
graduation,0.663764,0.25115,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
ability,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.464335,0.373262,0.0,0.0,0.0
academic,0.000000,0.00000,0.0,0.0,0.0,0.0,0.558299,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
added,0.000000,0.00000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0


In [16]:
# pca = PCA(n_components=2) 
# reduced_data = pca.fit_transform(vectorized_documents.toarray()) 

In [17]:
kmeans = cluster.KMeans(n_clusters = 9
                       , init = 'k-means++'
                       , n_init = 10
                       , tol = 0.0001
                       , random_state = 1)

# num_clusters = 9
# kmeans = KMeans(n_clusters=num_clusters, n_init=5, 
#                 max_iter=500, random_state=42) 
kmeans.fit(vectorized_documents) 

results = pd.DataFrame() 
results['document'] = lines
results['cluster'] = kmeans.labels_ 
  
# print the results 
print(results) 

                                             document  cluster
0             what are your plans after graduation?\n        7
1   would you like to provide a non-lmu email so w...        7
2   student-faculty relationship and department en...        2
3   your relation and interaction with faculty in ...        8
4   student-student relationships in the department\n        2
5                   advising: how do you feel about\n        3
6          academic program advising in mathematics\n        3
7                     the way advisors are assigned\n        2
8                                   career advising\n        3
9          advising on personal matters if relevant\n        3
10  lower-division experience: what are your thoug...        0
11              lower division course work in math \n        4
12  the level of difficulty of lower division math...        4
13        transition from high school to university\n        2
14  non-course experiences, e.g. dorms, added free...  

In [18]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = vectorizer.get_feature_names_out()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs

# def plotWords(dfs, n_feats):
#     plt.figure(figsize=(8, 4))
#     for i in range(0, len(dfs)):
#         plt.title(("Most Common Words in Cluster {}".format(i)), fontsize=10, fontweight='bold')
#         sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[i][:n_feats])
#         plt.show()

In [19]:
# best_result = 5
# kmeans = kmeans_results.get(best_result)

final_df_array = final_df.to_numpy()
prediction = kmeans.predict(final_df)
n_feats = 3
dfs = get_top_features_cluster(final_df_array, prediction, n_feats)
print(dfs)

[     features     score
0  experience  0.443739
1     thought  0.391985
2    division  0.271734,   features     score
0     unit  0.621517
1   course  0.430851
2   versus  0.295190,        features     score
0       student  0.296206
1    department  0.216054
2  relationship  0.190688,    features     score
0  advising  0.540543
1    career  0.200416
2      feel  0.164599,    features     score
0  division  0.384394
1      math  0.329164
2    course  0.279689,       features     score
0      ability  0.418798
1   confidence  0.418798
2  mathematics  0.381115,    features  score
0   comment    1.0
1   writing    0.0
2  decision    0.0,      features     score
0  graduation  0.304971
1         lmu  0.262803
2        plan  0.249314,      features     score
0  department  0.469863
1  suggestion  0.410567
2    relation  0.265439]


/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(
